In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd , sys
import matplotlib.pyplot as plt
import scikitplot as skplt
from IPython.display import clear_output
from sklearn.metrics import auc , roc_auc_score
sys.path.append("/home/advice/Python/SR/Custom/")
from RAdam import RAdamOptimizer
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import re , os
import tensorflow as tf

In [2]:
import optuna
from optuna.integration import SkoptSampler
import logging
import plotly
from optuna.logging import get_logger
from optuna.structs import TrialState
from optuna import type_checking
from optuna.visualization.utils import _check_plotly_availability
from optuna.visualization.utils import is_available
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ["CUDA_VISIBLE_DEVICES"]="1"
tf.logging.set_verbosity(tf.logging.ERROR)

/root/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning:

sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.



In [3]:
from tensorflow.contrib.layers import *
def get_weight_variable(shape, name=None,
                        type='xavier_uniform', regularize=True, **kwargs):
    initialise_from_constant = False
    if type == 'xavier_uniform':
        initial = xavier_initializer(uniform=True, dtype=tf.float32)
    elif type == 'xavier_normal':
        initial = xavier_initializer(uniform=False, dtype=tf.float32)
    elif type == 'he_normal':
        initial = variance_scaling_initializer(uniform=False, factor=2.0, mode='FAN_IN', dtype=tf.float32)
    elif type == 'he_uniform':
        initial = variance_scaling_initializer(uniform=True, factor=2.0, mode='FAN_IN', dtype=tf.float32)
    elif type == 'caffe_uniform':
        initial = variance_scaling_initializer(uniform=True, factor=1.0, mode='FAN_IN', dtype=tf.float32)
    elif type == 'simple':
        stddev = kwargs.get('stddev', 0.02)
        initial = tf.truncated_normal(shape, stddev=stddev, dtype=tf.float32)
        initialise_from_constant = True
    elif type == 'bilinear':
        weights = _bilinear_upsample_weights(shape)
        initial = tf.constant(weights, shape=shape, dtype=tf.float32)
        initialise_from_constant = True
    else:
        raise ValueError('Unknown initialisation requested: %s' % type)

    if name is None:  # This keeps to option open to use unnamed Variables
        weight = tf.Variable(initial)
    else:
        if initialise_from_constant:
            weight = tf.get_variable(name, initializer=initial)
        else:
            weight = tf.get_variable(name, shape=shape, initializer=initial)
    if regularize:
        tf.add_to_collection('weight_variables', weight)
    return weight 

In [4]:
data = pd.read_csv("./../Data/income_evaluation.csv")
objcol = data.select_dtypes("object").columns.tolist()
data[objcol] = data[objcol].astype("category")
data.columns = [i.strip() for i in data.columns.tolist()]

In [5]:
data["income"] = data["income"].cat.codes
target = data.pop("income")
num_col = data.select_dtypes("int").columns.tolist()
cat_col = data.select_dtypes("category").columns.tolist()
onehot_data = pd.get_dummies(data , columns= cat_col)

In [6]:
from sklearn.model_selection import train_test_split
Train_X , Test_X , Train_y , Test_y = train_test_split(onehot_data , 
                                                         target , test_size = 0.3)

In [7]:
one_hot_var = Train_X.columns.tolist()
in_var = data.columns.tolist()

In [8]:
def spectral_norm(w, iteration=1 , name = None):
    w_shape = w.shape.as_list()
    w = tf.reshape(w, [-1, w_shape[-1]])
    u = tf.get_variable(name , [1, w_shape[-1]], 
                        initializer=tf.random_normal_initializer(), trainable=False)
    u_hat = u
    v_hat = None
    for i in range(iteration):
        v_ = tf.matmul(u_hat, tf.transpose(w))
        v_hat = tf.nn.l2_normalize(v_)

        u_ = tf.matmul(v_hat, w)
        u_hat = tf.nn.l2_normalize(u_)

    u_hat = tf.stop_gradient(u_hat)
    v_hat = tf.stop_gradient(v_hat)

    sigma = tf.matmul(tf.matmul(v_hat, w), tf.transpose(u_hat))

    with tf.control_dependencies([u.assign(u_hat)]):
        w_norm = w / sigma
        w_norm = tf.reshape(w_norm, w_shape)
    return w_norm 

def OneHotIndex(in_var , num_var , one_hot_var) :
    start_idx = 0
    key_store = {}
    store = []
    for idx , col in enumerate(in_var) :
        if col in num_var :
            aa = [start_idx , start_idx +1]
            store.append(aa)
            start_idx += 1
        else :
            find = [idx for idx , ck in enumerate(one_hot_var) if re.search("^{}_".format(col) , ck)]
            nn = len(find)
            aa = [start_idx , start_idx + nn]
            start_idx += nn
            store.append(aa)
        key_store[col] = aa
    return key_store , store

In [9]:
import re
key_onehot_store , onehot_store = OneHotIndex(in_var , num_col , one_hot_var)
Train_y.reset_index(drop=True ,inplace=True)
target_1_list = Train_y[(Train_y == 1) == True].index.tolist()
target_0_list = Train_y[(Train_y == 0) == True].index.tolist()

In [10]:
ck = list(set(target_1_list) & set(target_0_list))
assert ck == [], "중복 발생 : {}".format(ck)

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

numeric_features = num_col 
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)])
clf = Pipeline(steps=[('preprocessor', preprocessor),])
clf.fit(Train_X)
Train_X[num_col] = clf.transform(Train_X)
#Valid_X[num_col] = clf.transform(Valid_X)
Test_X[num_col] = clf.transform(Test_X)

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/root/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.

In [12]:
Train_X_np = Train_X.values
#Valid_X_np = Valid_X.values
Test_X_np = Test_X.values
print(in_var)

['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']


In [13]:
len(in_var)

14

In [ ]:
def variable_select(in_var = None , method = None , select_n = None , NTree = None) :
    var_n = len(in_var)
    if method== "sqrt" :
        value =np.sqrt(var_n)
    elif method == "log2" :
        value =np.log2(var_n)
    elif method == "select" :
        value = select_n
    else :
        value = var_n
    return [list(np.random.choice(in_var ,
                                  replace = False , 
                                  size = value)) for _ in range(NTree)]
select_var = variable_select(in_var=in_var , method="select" , 
                               select_n= 10 , NTree= 5)
print(select_var)

[['hours-per-week', 'age', 'education-num', 'marital-status', 'occupation', 'workclass', 'sex', 'education', 'relationship', 'fnlwgt'], ['education', 'capital-gain', 'relationship', 'marital-status', 'age', 'fnlwgt', 'native-country', 'hours-per-week', 'race', 'education-num'], ['marital-status', 'capital-gain', 'relationship', 'capital-loss', 'sex', 'occupation', 'native-country', 'education-num', 'race', 'age'], ['race', 'native-country', 'sex', 'capital-loss', 'marital-status', 'fnlwgt', 'education-num', 'age', 'hours-per-week', 'occupation'], ['occupation', 'native-country', 'workclass', 'fnlwgt', 'education-num', 'relationship', 'capital-loss', 'marital-status', 'race', 'sex']]


## 앙상블 테스트

In [ ]:
def tf_mish(x) :
    act = x * tf.nn.tanh(tf.nn.softplus(x))
    return act

In [ ]:
activate_candidate = \
[tf.nn.selu, tf_mish , tf.nn.leaky_relu , tf.nn.elu ]

In [ ]:
from optuna import trial

## [optuna](https://github.com/optuna/optuna/blob/master/examples/tensorflow_eager_simple.py)

In [ ]:
Train_X_np.shape
## 

(22792, 108)

In [ ]:
import numpy as np
list(np.arange(8,14))


[8, 9, 10, 11, 12, 13]

In [ ]:

def create_model(Nensemble = select_var , trial = None) :
    layers_1 = trial.suggest_int("layers_1",2,4)
    layers_2 = trial.suggest_int("layers_2",2,4)
    layers_3 = trial.suggest_int("layers_3",2,4)
    layers_4 = trial.suggest_int("layers_4",2,4)
    layers_5 = trial.suggest_int("layers_5",2,4)
    nodes_1 = trial.suggest_int("nodes_1",30,65)
    nodes_2 = trial.suggest_int("nodes_2",30,65)
    nodes_3 = trial.suggest_int("nodes_3",30,65)
    nodes_4 = trial.suggest_int("nodes_4",30,65)
    nodes_5 = trial.suggest_int("nodes_5",30,65)
    
    init_candidate =\
    ["xavier_uniform","xavier_normal", 
     "he_normal", "he_uniform","caffe_uniform"]
    activate_candidate = \
    [tf.nn.selu, tf_mish , tf.nn.leaky_relu , tf.nn.elu , tf.nn.relu ]
    activate_int = [0,1,2,3,4]
    Ensembles = []
    target_n = 2 
    hidden = [
        [nodes_1] * layers_1 + [target_n] , 
        [nodes_2] * layers_2 + [target_n] ,
        [nodes_3] * layers_3 + [target_n] ,
        [nodes_4] * layers_4 + [target_n] ,
        [nodes_5] * layers_5 + [target_n]
    ]
    
    init =\
    trial.suggest_categorical('weight_init', init_candidate)
    row , dim = Train_X_np.shape
    selected =\
    trial.suggest_categorical('activation', activate_int)
    activation = activate_candidate[selected]
    X = tf.placeholder(tf.float32, shape = [ None , dim],name="X")
    y = tf.placeholder(tf.float32, shape = [ None , 1], name="y")
    DropoutRate = tf.placeholder(tf.float32, name ="dropoutRate")
    training = tf.placeholder_with_default(False, shape=[], name="training")
    with tf.variable_scope(f'Ensembles_{trial.number}' , reuse = tf.AUTO_REUSE):
        matrix = []
        for idx , __vars__ in enumerate(Nensemble) :
            with tf.variable_scope(f"Tree{idx}" , reuse = tf.AUTO_REUSE) :
                x_input = []
                for key in __vars__ :
                    start_node , terminal_node = key_onehot_store[key]
                    diff = terminal_node - start_node
                    X_Split = tf.slice(X , [0, start_node] , [-1 , diff],
                                       name = f"Tree{idx}_{key}")
                    x_input.append(X_Split)
                x_input = tf.concat(x_input , axis = 1, name = f"Tree{idx}_Concat")
                TOTAL_DIM = x_input.get_shape().as_list()[1]
                dims = [TOTAL_DIM] + hidden[idx]
                matrix.append(dims)
                for idx2 , h_dim in enumerate(dims) :
                    if idx2 == 0 :
                        Weight =get_weight_variable(shape = [TOTAL_DIM , h_dim], 
                                                    name=f"W_{idx}{idx2}",
                                                    type=init, regularize=True)
                        Bias = tf.get_variable(f"BIAS_{idx}{idx2}",
                                               shape = [h_dim] , dtype = tf.float32 , 
                                               initializer = tf.constant_initializer(0.0))
                        tf.summary.histogram(f"Weight{idx2}", Weight)
                        tf.summary.histogram(f"Bias{idx2}", Bias)
                        
                        Layer = tf.matmul( x_input , Weight) + Bias
                        Layer = activation(Layer)
                        Weight =get_weight_variable(shape = [TOTAL_DIM , 10], 
                                                    name=f"W_Info_{idx}{idx2}",
                                                    type=init, regularize=True)
                        InfoLayer = tf.matmul( x_input , Weight)
                        Layer = tf.contrib.nn.alpha_dropout(Layer , DropoutRate ) 
                    else :
                        if idx2 == 1 : 
                            h_n = dims[idx2-1] + 10
                        else : 
                            h_n = dims[idx2-1]
                        Weight =get_weight_variable(shape = [h_n ,h_dim ], 
                                                    name=f"W_{idx}{idx2}",
                                                    type=init, regularize=True)
                        Bias = tf.get_variable(f"BIAS_{idx}{idx2}",
                                               shape = [h_dim] , dtype = tf.float32 , 
                                               initializer = tf.constant_initializer(0.0))
                        if idx2 == 1 : 
                            Layer = tf.concat([Layer, InfoLayer], axis = 1)
                        else : pass
                        if len(dims) == idx2+1 : 
                            Layer = tf.matmul( Layer , Weight) + Bias
                        else : 
#                             Weight = spectral_norm(Weight , name = f"SNW_{idx}{idx2}")
                            Layer = tf.matmul( Layer , Weight) + Bias
                            Layer = activation(Layer)
                            Layer = tf.contrib.nn.alpha_dropout(Layer , DropoutRate ) 
                        tf.summary.histogram(f"Weight{idx2}", Weight)
                        tf.summary.histogram(f"Bias{idx2}", Bias)
            Ensembles.append(Layer)
        
        NModels = Ensembles
        y_one_hot =\
        tf.one_hot( 
            tf.cast(
                tf.squeeze(y , axis = 1 ) , tf.int32) , depth= target_n)
        
        y_weight_info = compute_class_weight(class_weight= "balanced" , 
                             classes = np.unique(target),
                             y= np.squeeze(target))
        weight = tf.constant([ y_weight_info[1] ] ) # 
        mod = sys.modules[__name__]
        for idx , Model in enumerate(NModels) :
            setattr(mod, 'model_{}_softmax'.format(idx), 
                    tf.argmax( tf.nn.softmax(Model) , axis = 1 ))
        Loss = []
        Probs = 0
        for idx , Model in enumerate(NModels) :
            loss = tf.nn.weighted_cross_entropy_with_logits(targets = y_one_hot ,
                                                             logits = Model , 
                                                             pos_weight = weight)
            Probs +=tf.nn.softmax(Model)
            Loss.append(loss)
            #Loss += loss
        Loss = tf.reduce_mean(Loss)
        #Loss /= len(NModels)
        Probs = tf.nn.softmax(Probs)
        tf.summary.histogram(f"Probability", Probs)
        vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="Ensembles")
        L2 = []
        WEIGHTS = []
        
        for v in vars :
            if re.search('W_' , v.name) :
                WEIGHTS.append(v)
                L2.append(tf.nn.l2_loss(v))
        Loss += tf.add_n(L2)  * 0.01
        l1_regularizer = tf.contrib.layers.l1_regularizer(scale=0.005, 
                                                          scope=None)
        regularization_penalty = tf.contrib.layers.apply_regularization(l1_regularizer, 
                                                                        WEIGHTS )
        Loss += regularization_penalty
        kwargs = {}
#         optimizer_options = ['RMSPropOptimizer', 'AdamOptimizer', 'MomentumOptimizer',"RADAM"]
#         optimizer_selected = trial.suggest_categorical('optimizer', optimizer_options)
#         if optimizer_selected == 'RMSPropOptimizer':
#             kwargs['learning_rate'] = trial.suggest_loguniform('rmsprop_learning_rate', 1e-5, 1e-1)
#             kwargs['decay'] = trial.suggest_uniform('rmsprop_decay', 0.85, 0.99)
#             kwargs['momentum'] = trial.suggest_loguniform('rmsprop_momentum', 1e-5, 1e-1)
#             optimizer = getattr(tf.train, optimizer_selected)(**kwargs)
#         elif optimizer_selected == 'AdamOptimizer':
#             kwargs['learning_rate'] = trial.suggest_loguniform('adam_learning_rate', 1e-5, 1e-1)
#             optimizer = getattr(tf.train, optimizer_selected)(**kwargs)
#         elif optimizer_selected == 'MomentumOptimizer':
#             kwargs['learning_rate'] = trial.suggest_loguniform('momentum_opt_learning_rate',
#                                                                1e-5,
#                                                                1e-1)
#             kwargs['momentum'] = trial.suggest_loguniform('momentum_opt_momentum', 1e-5, 1e-1)
#             optimizer = getattr(tf.train, optimizer_selected)(**kwargs)
#         else :
        kwargs['learning_rate'] = trial.suggest_loguniform('Radam_learning_rate', 1e-5, 1e-1)
        optimizer = RAdamOptimizer(**kwargs)
        solver = optimizer.minimize(Loss ,var_list = vars )
        
        prediction = tf.argmax(Probs, 1)
        correct = tf.argmax(y_one_hot, 1)
        equality = tf.equal(prediction, correct)
        accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))
        tf.summary.scalar(f"Accuracy_{trial.number}",accuracy)
        tf.summary.scalar(f"Loss_{trial.number}",Loss)
        merged = tf.summary.merge_all()
        logging.getLogger().info("Try : {} \n {}".format(trial.number,
                                                     str(matrix)))
        return [Ensembles , Probs , Loss , solver, 
                X , y , DropoutRate , training,merged]
    

In [ ]:
best_auc = 0.0
def objective(trial):   
#     selected =\
#     trial.suggest_categorical('select_n', np.arange(8,14).astype(int).tolist())
#     select_var = variable_select(in_var=in_var , 
#                                  method="select" , 
#                                  select_n= selected , NTree= 5)
    Lists = create_model(Nensemble = select_var,
                         trial =trial)
    
    (NModels , Probs ,
     Loss , solver,
     X , y ,
     DropoutRate , training,
     merged
    ) = Lists
    Epoch = 500
    mb_size = 500
    
    
    config=tf.ConfigProto(
        log_device_placement=True,
#         intra_op_parallelism_threads=8,
#         inter_op_parallelism_threads=4
    )
    config.gpu_options.allow_growth = True
    sess = tf.Session(config = config)
    
    train_writer = tf.summary.FileWriter(f'./OptunaResult/train_{trial.number}',
                                         sess.graph)
    sess.run(tf.global_variables_initializer())
    _Loss_ = []
    _Epoch_ = []
    _Epoch2_ = [0]
    _trAUC_ , _teAUC_ = [0] , [0]
    
    
    for epoch in range(Epoch) :
        target1 = np.random.choice(target_1_list ,
                                   int(len(target_1_list)/2) ,
                                   replace = False )
        target0 = np.random.choice(target_0_list , 
                                   len(target_0_list) ,
                                   replace = True )
        target1 = list(target1)
        target0 = list(target0)
#         if epoch > 10 :
#             target1 = target1 + checkpoint 
        XX = Train_X_np[target1 + target0  , : ]
        YY = Train_y.values[target1 + target0]
        idx = np.random.permutation(len(XX))
        XX = XX[idx , : ]
        YY = YY[idx]
        batch_iter = int(len(XX) / mb_size)
        batchLoss = 0
        for idx in range(batch_iter) :
            X_mb = XX[idx*mb_size:(idx+1)*mb_size]
            Y_mb = YY[idx*mb_size:(idx+1)*mb_size]
            Feed = {X : X_mb ,
                    y : Y_mb.reshape(-1,1) , 
                    DropoutRate : 0.5 ,
                    training : True }
            _ , LOSS =\
            sess.run([solver , Loss] ,
                     feed_dict= Feed
                    )
            
            batchLoss += LOSS
        batchLoss /= batch_iter
        _Loss_.append(batchLoss)
        _Epoch_.append(epoch)
        
        
        if epoch % 10 == 9 :
#             run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
#             run_metadata = tf.RunMetadata()
            Feed = {X : X_mb ,
                    y : Y_mb.reshape(-1,1) , 
                    DropoutRate : 0.5 ,
                    training : True }
            _ , summary, LOSS =
            sess.run([solver , merged, Loss] ,
                     feed_dict= Feed,)
#                      options=run_options,
#                      run_metadata=run_metadata
#             train_writer.add_run_metadata(run_metadata, 
#                                           'epoch%03d' % epoch)
            train_writer.add_summary(summary, epoch)
            
        if epoch % 100 == 0 :
            Feed = {X : Train_X_np  ,
                    DropoutRate : 0.5 ,
                    training : True }
            probs  = sess.run(Probs , 
                              feed_dict= Feed,)
            real_target = np.squeeze(Train_y.values)
            pred_target = np.argmax(probs,axis =1)
            AUC = roc_auc_score(real_target , probs[:,1])
#             DD = pd.DataFrame([real_target,
#                                pred_target], 
#                               index = ["t","p"]).T
#             DD2 = DD[(DD.t == 1) & (DD.p==0)]
#             checkpoint = DD2.index.tolist()
            if epoch % 100 == 0 :
                msg = "Try : {:3}, Epoch : {:3}, AUC : {:.2f}%".\
            format(trial.number, epoch , 100*AUC)
                logging.getLogger().info(msg)
    train_writer.close()
    Feed = {X : Train_X_np  ,
            DropoutRate : 0.5 ,
            training : True }
    probs  = sess.run(Probs , feed_dict= Feed)
    real_target = np.squeeze(Train_y.values)
    pred_target = np.argmax(probs,axis =1)
    AUC = roc_auc_score(real_target , probs[:,1])
    
    global best_auc
    if best_auc < AUC :
        logging.getLogger().info("Try : {}, {} < {}".format(trial.number , best_auc ,AUC))
        best_auc= AUC
    tf.reset_default_graph()
    clear_output()
    return AUC

In [ ]:
logging.getLogger().setLevel(logging.INFO)
logging.getLogger().addHandler(logging.FileHandler('optuna.txt', mode="w"))
optuna.logging.enable_propagation()
optuna.logging.disable_default_handler()

In [ ]:

x0 = [[3, 2, 4, 3, 4, 36, 32, 49, 40, 35, 'he_uniform', 3, 0.00015671897437797408], 
      [3, 4, 2, 2, 3, 34, 64, 45, 54, 40, 'he_normal', 3, 1.4147691642608397e-05], 
      [4, 3, 3, 4, 2, 46, 65, 62, 42, 57, 'xavier_uniform', 0, 0.0001151939657356718], [3, 4, 2, 4, 4, 60, 34, 54, 63, 40, 'he_uniform', 3, 0.00023751936361984937], [11, 2, 4, 3, 4, 3, 38, 58, 39, 31, 63, 'caffe_uniform', 4, 1.185261441167407e-05], [12, 2, 2, 2, 2, 3, 43, 54, 42, 56, 30, 'caffe_uniform', 3, 2.8053303983617267e-05], [13, 3, 4, 3, 4, 2, 53, 31, 57, 31, 40, 'he_uniform', 1, 0.004342367249195597], [10, 3, 2, 2, 2, 2, 52, 56, 50, 40, 45, 'caffe_uniform', 2, 0.0002353657442834687], [12, 2, 3, 2, 2, 4, 37, 43, 38, 48, 56, 'xavier_normal', 2, 1.1695403014796976e-05], [10, 4, 3, 2, 4, 4, 62, 55, 56, 30, 44, 'xavier_uniform', 2, 0.00132122127636011], [12, 2, 4, 4, 3, 3, 64, 47, 60, 39, 64, 'xavier_normal', 3, 0.002818043271032938], [11, 2, 4, 3, 3, 4, 54, 34, 35, 49, 63, 'xavier_uniform', 1, 0.00011653454036311457], [12, 4, 2, 3, 4, 3, 51, 45, 36, 41, 40, 'caffe_uniform', 1, 0.0013094508034332806], [9, 3, 2, 4, 4, 3, 62, 51, 64, 36, 42, 'xavier_normal', 3, 0.006172462319995353], [8, 2, 3, 4, 2, 2, 65, 38, 38, 53, 65, 'xavier_uniform', 1, 0.004848434779819864], [8, 3, 2, 3, 2, 4, 65, 65, 60, 30, 30, 'caffe_uniform', 2, 0.09999999999999999], [10, 4, 4, 4, 3, 3, 65, 34, 65, 30, 46, 'xavier_normal', 3, 0.09999999999999999], [8, 2, 4, 3, 3, 4, 61, 53, 43, 52, 46, 'caffe_uniform', 3, 0.09999999999999999], [8, 4, 3, 4, 3, 2, 63, 57, 44, 55, 48, 'he_uniform', 1, 0.09999999999999999], [8, 2, 3, 4, 2, 3, 36, 30, 30, 63, 32, 'he_normal', 2, 0.09999999999999999], [8, 2, 3, 4, 2, 2, 60, 39, 41, 65, 33, 'xavier_uniform', 4, 0.0002012076028834959], [11, 3, 3, 4, 3, 4, 35, 63, 63, 62, 64, 'xavier_uniform', 1, 0.00866714545089891], [8, 2, 3, 3, 4, 4, 55, 56, 63, 60, 34, 'he_normal', 2, 0.00010933878687689759], [8, 2, 4, 3, 4, 4, 48, 40, 52, 60, 32, 'he_normal', 2, 0.09999999999999999], [8, 4, 4, 3, 3, 4, 60, 54, 65, 41, 32, 'he_normal', 3, 1.0054555508216113e-05], [8, 3, 3, 3, 3, 2, 31, 57, 60, 51, 30, 'he_normal', 1, 0.0026168135968490723], [8, 4, 4, 4, 3, 3, 37, 64, 62, 33, 52, 'he_normal', 0, 0.01685645874697779], [11, 4, 3, 3, 3, 4, 61, 47, 54, 55, 47, 'he_normal', 1, 0.0003765011074475807], [10, 2, 3, 3, 3, 3, 63, 52, 56, 58, 34, 'caffe_uniform', 3, 0.09999999999999999], [8, 2, 2, 4, 2, 2, 30, 30, 30, 65, 30, 'he_normal', 2, 0.09999999999999999], [8, 2, 2, 3, 2, 3, 30, 30, 30, 53, 32, 'he_normal', 0, 0.09999999999999999], [8, 4, 3, 3, 4, 3, 48, 63, 47, 34, 61, 'caffe_uniform', 0, 0.0006988342189973038], [8, 2, 2, 4, 2, 3, 48, 50, 38, 60, 44, 'he_normal', 3, 0.0010630558921353032], [8, 4, 3, 3, 3, 3, 55, 63, 37, 31, 51, 'xavier_uniform', 0, 0.040182145025863716], [8, 4, 3, 2, 4, 3, 52, 36, 65, 36, 54, 'caffe_uniform', 2, 1.799227078888316e-05], [8, 2, 2, 3, 3, 3, 30, 42, 46, 53, 53, 'he_normal', 3, 0.0017274481636200155], [8, 4, 2, 2, 4, 3, 39, 42, 58, 58, 34, 'caffe_uniform', 0, 0.02538727255814498], [8, 3, 2, 3, 2, 3, 34, 64, 54, 32, 44, 'caffe_uniform', 0, 0.05841467788043324], [8, 3, 2, 2, 4, 3, 30, 40, 36, 40, 35, 'caffe_uniform', 0, 2.6425141972924894e-05], [8, 4, 2, 3, 4, 3, 53, 50, 65, 54, 65, 'xavier_uniform', 2, 0.00035018555480494817], [8, 4, 2, 2, 4, 3, 60, 61, 64, 38, 56, 'xavier_uniform', 4, 0.00030709124496748716], [8, 2, 2, 3, 2, 3, 30, 53, 53, 56, 64, 'he_normal', 3, 0.056749864882203925], [8, 2, 2, 4, 2, 3, 30, 65, 59, 30, 65, 'xavier_uniform', 0, 0.09999999999999999], [9, 2, 2, 4, 3, 3, 57, 60, 56, 50, 63, 'xavier_uniform', 1, 2.1659583232534686e-05], [13, 2, 2, 4, 2, 3, 30, 65, 65, 30, 62, 'he_normal', 0, 0.007582478981301236], [8, 2, 2, 4, 2, 3, 30, 65, 65, 30, 65, 'he_normal', 0, 0.09999999999999999], [9, 3, 2, 4, 2, 3, 45, 65, 65, 30, 30, 'he_normal', 2, 0.09999999999999999], [8, 2, 2, 4, 2, 3, 41, 65, 30, 30, 65, 'he_normal', 2, 0.011195733193718348], [8, 2, 2, 4, 2, 3, 65, 65, 43, 30, 37, 'he_normal', 2, 0.015092488426198855], [8, 4, 2, 3, 2, 3, 63, 65, 47, 34, 43, 'he_normal', 2, 1e-05], [8, 4, 2, 2, 3, 3, 57, 55, 65, 42, 65, 'he_normal', 2, 1e-05], [8, 2, 3, 3, 4, 3, 34, 62, 47, 38, 64, 'he_normal', 0, 1.0017455593748346e-05], [10, 3, 2, 4, 2, 3, 54, 65, 39, 46, 50, 'he_normal', 0, 0.0010848883882730042], [13, 2, 3, 2, 2, 3, 57, 65, 36, 56, 55, 'he_normal', 0, 1e-05], [12, 3, 2, 2, 2, 3, 65, 53, 65, 35, 30, 'he_normal', 0, 1e-05], [13, 4, 3, 2, 4, 3, 47, 43, 53, 37, 61, 'he_normal', 4, 2.3448490928467227e-05], [8, 4, 3, 2, 2, 3, 43, 42, 31, 51, 48, 'caffe_uniform', 0, 8.923072180908639e-05], [13, 2, 2, 2, 3, 3, 36, 65, 30, 33, 64, 'he_normal', 0, 2.5008788649091418e-05], [13, 2, 3, 4, 2, 3, 55, 34, 41, 41, 33, 'he_normal', 0, 6.300417507628337e-05], [13, 3, 3, 4, 2, 3, 46, 49, 36, 36, 59, 'he_normal', 0, 1.4168106849205638e-05], [13, 3, 3, 4, 2, 3, 44, 57, 51, 57, 44, 'he_normal', 0, 0.047648161089890124], [10, 3, 3, 4, 2, 3, 53, 58, 48, 63, 52, 'he_normal', 0, 0.0006616156670531221], [11, 3, 3, 2, 2, 3, 49, 56, 61, 50, 35, 'he_normal', 0, 0.003874206631357768], [11, 4, 3, 2, 4, 3, 30, 35, 49, 30, 48, 'he_normal', 0, 0.09999999999999999], [13, 2, 3, 2, 4, 3, 45, 65, 30, 40, 39, 'caffe_uniform', 0, 1.3124990909659663e-05], [11, 4, 3, 2, 2, 3, 59, 60, 36, 51, 65, 'xavier_uniform', 0, 0.016798014117541147], [13, 4, 3, 2, 2, 3, 59, 51, 37, 48, 65, 'xavier_uniform', 0, 0.00010245200147978441], [13, 4, 4, 2, 2, 3, 56, 65, 30, 30, 65, 'he_normal', 0, 1e-05], [8, 4, 4, 2, 2, 3, 32, 58, 60, 39, 51, 'he_normal', 0, 1.138181205622081e-05], [8, 4, 2, 2, 2, 3, 41, 54, 30, 30, 65, 'he_normal', 0, 1e-05], [8, 4, 2, 2, 2, 3, 32, 55, 30, 30, 65, 'he_normal', 0, 1e-05], [13, 3, 3, 2, 2, 3, 38, 48, 56, 36, 65, 'caffe_uniform', 2, 7.267730586124988e-05], [13, 3, 3, 2, 2, 3, 55, 58, 42, 34, 65, 'he_normal', 2, 1.960429089732139e-05], [8, 4, 4, 3, 2, 3, 46, 61, 48, 36, 38, 'he_normal', 2, 0.03533183513155234], [8, 4, 4, 3, 2, 3, 45, 40, 64, 60, 41, 'he_normal', 1, 0.0017356418135645042], [8, 3, 4, 3, 2, 3, 58, 43, 47, 38, 40, 'he_normal', 0, 0.0009185683862714557], [8, 3, 4, 3, 2, 3, 62, 51, 57, 40, 42, 'he_normal', 0, 0.0003828493855941364]]
y0 = [0.5, 0.37312627079078414, 0.5, 0.5, 0.40127449514370905, 0.5, 0.5, 0.5, 0.3953523854056861, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.33482347489277114, 0.5194659099702321, 0.1966741884180666, 0.42179660969198135, 0.5488758740195663, 0.5, 0.5, 0.5, 0.4357786121775885, 0.32259930095007106, 0.5, 0.5, 0.5, 0.31408737435719747, 0.3165046787537358, 0.5782271547251162, 0.5, 0.5, 0.5000287786347415, 0.5, 0.5, 0.5, 0.5033644497408181, 0.5, 0.5, 0.5, 0.4474929984736806, 0.45493217463262253, 0.5, 0.5, 0.5785984352326466, 0.31545481099947675, 0.5, 0.5, 0.5561970460903911, 0.3749490185795035, 0.439759109548035, 0.5, 0.6167678019492814, 0.28535717674560535, 0.5, 0.5, 0.5, 0.5, 0.5611044938310887, 0.5258427040774896, 0.5, 0.5, 0.24205781919861497, 0.5, 0.5000913182518414, 0.5, 0.38260363612366477, 0.700793203550933, 0.507285148316521, 0.4964040570034959, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]

sampler = SkoptSampler(
    skopt_kwargs={'n_random_starts':0,
                  'acq_func':'EI',
                  'acq_func_kwargs': {'xi':0.02, 
                                      "x0" : None, 
                                      "y0" : None}})
#optuna.logging.enable_default_handler()

In [ ]:
if __name__ == '__main__':
    logging.getLogger().info("Start optimization.")
    study = optuna.create_study(direction='maximize',
                               sampler = sampler)
    study.optimize(objective, 
                   n_trials=100,
                   n_jobs = 5
                  )
    print('Number of finished trials: ', len(study.trials))

    print('Best trial:')
    trial = study.best_trial

    print('  Value: ', trial.value)

    print('  Params: ')
    for key, value in trial.params.items():
        print('    {}: {}'.format(key, value))

I0126 13:51:07.922246 140701369960192 study.py:628] Finished trial#2 resulted in value: 0.5011268274490706. Current best value is 0.5011268274490706 with parameters: {'layers_1': 2, 'layers_2': 2, 'layers_3': 3, 'layers_4': 4, 'layers_5': 3, 'nodes_1': 58, 'nodes_2': 61, 'nodes_3': 36, 'nodes_4': 41, 'nodes_5': 42, 'weight_init': 'he_normal', 'activation': 4, 'Radam_learning_rate': 0.0027976533870262537}.
I0126 13:51:13.657022 140701369960192 <ipython-input-20-261486b73ed5>:169] Try : 5 
 [[59, 63, 63, 63, 63, 2], [81, 30, 30, 30, 30, 2], [81, 61, 61, 61, 61, 2], [76, 37, 37, 2], [89, 36, 36, 2]]


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: TITAN X (Pascal), pci bus id: 0000:04:00.0, compute capability: 6.1



I0126 13:51:24.960213 140701369960192 <ipython-input-21-1700dc0c0155>:107] Try :   5, Epoch :   0, AUC : 48.83%


In [ ]:
study.trials_dataframe().sort_values(["value"],ascending = False)

In [ ]:
x0 = []
y0 = []
for i in range(len(study.get_trials())) :
    check = study.get_trials()[i].value
    if check is None :
        continue
    x0.append(list(study.get_trials()[i].params.values()))
    y0.append(study.get_trials()[i].value)


In [ ]:
print(x0)
print(y0)

In [ ]:
if is_available():
    from optuna.visualization.plotly_imports import go

#optuna.visualization.plot_intermediate_values(study)

In [ ]:
#optuna.visualization.plot_optimization_history(study)

In [ ]:
#optuna.visualization.plot_contour(study, params=['layers_1', 'layers_2'])

In [ ]:
#optuna.visualization.plot_slice(study, params['layers_1', 'layers_2'])

In [ ]:
#optuna.visualization.plot_parallel_coordinate(study, params=['layers_1', 'layers_2'])